<a href="https://colab.research.google.com/github/zhousanfu/machine-learning-demo/blob/master/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain faiss-cpu
!pip install transformers sentence_transformers sentencepiece cpm_kernels
!pip install google-search-results -i pypi.douban.com/simple --trusted-host pypi.douban.com

## Chatglm

In [ ]:
from typing import Any, List, Mapping, Optional
from transformers import AutoTokenizer, AutoModel
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
import torch



class chatGLM():
    def __init__(self, model_name, quantization_bit=4) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if torch.cuda.is_available():
            self.model = AutoModel.from_pretrained(model_name, trust_remote_code=True).half().cuda().eval()
        else:
            self.model = model = AutoModel.from_pretrained(model_name, trust_remote_code=True).float()
        self.model = self.model.quantize(quantization_bit)

    def __call__(self, prompt, history=None) -> Any:
        response, history = self.model.chat(self.tokenizer , prompt, history=history) # 这里演示未使用流式接口. stream_chat()
        return response, history

llm = chatGLM(model_name="THUDM/chatglm-6B-int4")

In [3]:
response, history = llm(prompt="你好", history=[])
print("response: %s"%response)
response, history = llm(prompt="我最近有点失眠怎么办?", history=history)
print("response: %s"%response)

response: 你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。
response: 失眠的话，建议尝试下述方法进行改善：
1. 睡前放松：在睡前半小时尽量避免进行刺激性的活动，放松身心，例如泡个热水澡、听轻柔的音乐等。
2. 建立规律的睡眠时间：保持每天固定的睡眠时间和起床时间，帮助身体建立一个健康的睡眠节律。
3. 改善睡前环境：例如调暗灯光、保持安静、减少噪音等，或者尝试使用放松性质的布置，如舒适、柔软的床上用品等。
4. 避免咖啡因和酒精：这些物质会影响睡眠，尽量避免摄入。
5. 锻炼身体：适度的运动可以提高身体的代谢水平，帮助入睡。
如果上述方法无法改善失眠，建议咨询医生或专业的医疗机构，寻求更进一步的帮助。


## Langchain

LangChain是一个强大的框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源

In [4]:
# # 官方llm使用OPENAI 接口
# from langchain.llms import OpenAI
# llm = OpenAI(model_name="text-davinci-003")
# prompt = "你好"
# response = llm(prompt)

### google-search

In [ ]:
# from langchain.agents import load_tools
# import os


# os.environ["SERPAPI_API_KEY"] = '你的api key'

# tools = load_tools(["serpapi", "python_repl", "llm-math"], llm=llm)
# agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# agent.run("当前金价是多少一克？")

### prompt 提示
填入内容来引导大模型输出

In [8]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

template = """什么是{query},还有如何真正做到并细说实现步骤"""
prompt_tem = PromptTemplate(input_variables=["query"], template=template)
prompt = prompt_tem.format(query='阶级跳跃')

response, history = llm(prompt=prompt, history=[])
print(prompt, '-->', response)

什么是阶级跳跃,还有如何真正做到并细说实现步骤 --> 阶级跳跃是指在某个游戏中，玩家通过积累特定的道具和技能，从普通玩家变成高级玩家的过程。这种过程需要玩家在游戏中不断挑战自己，提高自己的技能和道具的利用效率，从而获得更多的游戏奖励和成就。

要实现阶级跳跃，需要玩家掌握以下技能和策略：

1. 道具的使用：玩家需要学会如何有效地使用游戏中的各种道具，如技能、装备、金币等，以提高自己的战斗力和属性。

2. 技能的学习：玩家需要学习游戏中的各种技能，如战斗技能、操作技能、策略技能等，以便在游戏中更好地发挥自己的能力。

3. 挑战和升级：玩家需要不断挑战游戏中的各种难度模式，如普通、专家、大师等，以提高自己的技能和道具的等级。

4. 金币的使用：玩家需要学会如何有效地使用游戏中的金币，以便购买更多的道具和技能，提高自己的战斗力和属性。

以下是实现阶级跳跃的具体步骤：

1. 学习技能：玩家需要学习游戏中的各种技能，并了解每个技能的作用和优点，以便更好地利用这些技能来提高自己的战斗力和属性。

2. 挑战难度模式：玩家需要不断挑战游戏中的各种难度模式，如普通、专家、大师等，以提高自己的技能和道具的等级。

3. 购买道具和技能：玩家需要购买游戏中的各种道具和技能，并了解它们的作用和优点，以便更好地利用这些道具和技能来提高自己的战斗力和属性。

4. 使用金币：玩家需要学会如何有效地使用游戏中的金币，以便购买更多的道具和技能，提高自己的战斗力和属性。

5. 持续学习和提高：玩家需要不断学习和提高自己的技能和道具的等级，以便在游戏中更好地发挥自己的能力，实现阶级跳跃。


In [9]:
system_template = "你是一个把{input_language}翻译成{output_language}的助手"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
messages = chat_prompt.format_prompt(input_language="英语", output_language="汉语", text="I love programming.")

messages.to_messages()

[SystemMessage(content='你是一个把英语翻译成汉语的助手', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

### Chains 链
链接多个组件处理一个特定的下游任务

In [ ]:
# from langchain.chains import LLMChain
# chain = LLMChain(llm=openAI(), prompt=promptTem)
# print(chain.run("你好"))

from langchain.chains.base import Chain



class DemoChain():
    def __init__(self, llm, prompt, history) -> None:
        self.llm = llm
        self.prompt = prompt
        self.history = history

    def run(self, query, history, context=None) -> Any:
        if context is not None:
            prompt = self.prompt.format(query=query, context=context)
        else:
            prompt = self.prompt.format(query=query)

        response, history = self.llm(prompt, history)
        return response, history

chain = DemoChain(llm=llm, prompt=prompt_tem, history=[])
response, history = chain.run(query="阶级跳跃", history=[])
print(response, history)

chain = DemoChain(llm=llm, prompt=prompt_tem, history=[])
response, history = chain.run(query="阶级跳跃", history=[])
print(response, history)

阶级跳跃是指一个人或组织通过不断努力和学习，从一个阶级跨越到另一个阶级的过程。这个过程通常涉及到对社会结构和游戏规则的深刻理解，以及在适当的时间和场合下采取适当的行动。

实现阶级跳跃的步骤如下：

1. 学习游戏规则和历史经验：阶级跳跃需要对游戏规则和历史经验有深入的了解。玩家需要学习如何在游戏中获得优势，如何赢得比赛，以及如何与对手竞争。

2. 建立强大的关系网络：关系网络可以帮助玩家获得游戏中的优势。玩家需要与其他人建立联系，以获得他们的信任和资源。

3. 提高自己的技能：玩家需要不断提高自己的技能，以增加在游戏中的优势。这可能包括学习新技能、获得更好的工作经验或获得更高的学历。

4. 选择适当的时间和场合：阶级跳跃需要在适当的时间和场合下进行。玩家需要选择适当的时间进行阶级跳跃，例如在赢得比赛之前进行阶级跳跃，或者在比赛中获得优势之后进行阶级跳跃。

5. 实施阶级跳跃：玩家需要采取适当的行动来实施阶级跳跃。这可能包括建立强大的关系网络、提高自己的技能、选择适当的时间和场合，以及实施阶级跳跃。

阶级跳跃是一个需要不断学习和改进的过程，也是一个需要耐心和毅力的过程。只有对于那些具有坚定的信念和毅力的人来说，阶级跳跃才能真正实现。 [('什么是阶级跳跃,还有如何真正做到并细说实现步骤', '阶级跳跃是指一个人或组织通过不断努力和学习，从一个阶级跨越到另一个阶级的过程。这个过程通常涉及到对社会结构和游戏规则的深刻理解，以及在适当的时间和场合下采取适当的行动。\n\n实现阶级跳跃的步骤如下：\n\n1. 学习游戏规则和历史经验：阶级跳跃需要对游戏规则和历史经验有深入的了解。玩家需要学习如何在游戏中获得优势，如何赢得比赛，以及如何与对手竞争。\n\n2. 建立强大的关系网络：关系网络可以帮助玩家获得游戏中的优势。玩家需要与其他人建立联系，以获得他们的信任和资源。\n\n3. 提高自己的技能：玩家需要不断提高自己的技能，以增加在游戏中的优势。这可能包括学习新技能、获得更好的工作经验或获得更高的学历。\n\n4. 选择适当的时间和场合：阶级跳跃需要在适当的时间和场合下进行。玩家需要选择适当的时间进行阶级跳跃，例如在赢得比赛之前进行阶级跳跃，或者在比赛中获得优势之后进行阶级跳跃。\n\n5. 实施阶级跳跃：玩家需要采取适当的行动来实施阶级跳跃。这可能包括建立强大的关

### Embedding
外部信息编码成一个高维向量

In [ ]:
# #官方示例代码，用的OpenAI的ada的文本Embedding模型
# #1） Embeding model
# from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model_name="ada")
# query_result = embeddings.embed_query("你好")

# #2) 文本切割
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100, chunk_overlap=0
# )
# texts = """阶级跳跃是指一个人或一个组织通过提高自己的技能、知识和领导能力，从一个阶级跨越到另一个阶级的过程。要实现阶级跳跃，以下是一些建议：\n\n1. 学习新技能：学习新技能可以让人具备新的知识和技能，从而增加自己的竞争力。可以选择学习与目前工作相关的新技能，或者学习与未来工作相关的新技能。\n\n2. 提高知识水平：不断学习新知识可以增加自己的知识储备，从而提高自己的竞争力。可以通过阅读书籍、参加培训、参与线上课程等方式来提高自己的知识水平。\n\n3. 建立良好的人际关系：建立良好的人际关系可以让人更容易得到新机会，同时也可以获得更多的支持和帮助。可以通过参加社交活动、建立人脉、参加社区组织等方式来建立良好的人际关系。\n\n4. 提高自己的领导能力：领导能力可以让人更好地管理自己的时间和资源，从而更好地完成工作。可以通过参加领导力课程、参加团队建设活动、自我反思等方式来提高自己的领导能力。\n\n5. 建立自己的品牌：建立自己的品牌可以让人更容易被人记住，从而更容易得到新机会。可以通过写博客、发布视频、制作网站等方式来建立自己的品牌。\n\n阶级跳跃需要长期的努力和不断学习，需要对自己的能力和目标有清晰的认识，并制定明确的计划和目标。"""
# texts = text_splitter.create_documents([texts])
# print(texts[0].page_content)

# # 3)入库检索，官方使用的Pinecone,他提供一个后台管理界面 | 用户需求太大，不好用了已经，一直加载中....
# import pinecone
# from langchain.vectorstores import Pinecone
# pinecone.init(api_key=os.getenv(""), enviroment=os.getenv(""))

# index_name = "demo"
# search = Pinecone.from_documents(texts=texts, embeddings, index_name=index_name)
# query = "What is magical about an autoencoder?"
# result = search.similarity_search(query)

### 这里使用chatGLM
# 1） Embedding model:  text2vec-large-chinese

In [ ]:
import numpy as np
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS


class TextSpliter(CharacterTextSplitter):
    def __init__(self, separator: str = "\n\n", **kwargs: Any):
        super().__init__(separator, **kwargs)

    def split_text(self, text: str) -> List[str]:
        texts = text.split("\n")
        texts = [Document(page_content=text, metadata={"from": "filename or book.txt"}) for text in texts]
        return texts

texts = response

text_splitter = TextSpliter()
texts = text_splitter.split_text(texts)
texts1 = [text.page_content for text in texts]

texts1

In [ ]:
model_name = "GanymedeNil/text2vec-large-chinese"
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={'device': "cuda"})
query_result = embeddings.embed_query("阶级跳跃")

np.array(query_result).shape

In [ ]:
vs_path = "text_to_emb"

docs = embeddings.embed_documents(texts1)

vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local(vs_path)

vector_store = FAISS.load_local(vs_path, embeddings)
related_docs_with_score = vector_store.similarity_search_with_score(query="阶级跳跃", k=2)

related_docs_with_score

In [ ]:
# 基于查询到的知识做prompt
context = ""
for pack in related_docs_with_score:
    doc, socre = pack
    content = doc.page_content
    print("检索到的知识=%s, from=%s, socre=%.3f"%(content, doc.metadata.get("from"), socre))
    context += content

In [ ]:
# 重新配置一个基于上下文的模板在来调下语言模型
template = "已知{context}, 请给我解释一下{query}的意思?"
promptTem = PromptTemplate(input_variables=["context", "query"], template=template)
chain = DemoChain(llm=llm, prompt=promptTem, history=[])
print("-"*80)
print(chain.run(query="阶级跳跃", context=context, history=[]))
print("-"*80)

### Embeddings HuggingFace

In [ ]:
##

### Embedding 持久化

In [ ]:
##

### 对超长文本进行总结

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import IPython
import sentence_transformers


embedding_model_dict = {
    "ernie-tiny": "nghuyong/ernie-3.0-nano-zh",
    "ernie-base": "nghuyong/ernie-3.0-base-zh",
    "text2vec": "GanymedeNil/text2vec-large-chinese",
    "text2vec2":"uer/sbert-base-chinese-nli",
    "text2vec3":"shibing624/text2vec-base-chinese",
}

EMBEDDING_MODEL = "text2vec3"
# 初始化 hugginFace 的 embeddings 对象
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_dict[EMBEDDING_MODEL], )
embeddings.client = sentence_transformers.SentenceTransformer(
        embeddings.model_name, device='mps')


In [ ]:
from langchain.vectorstores import Chroma
# 初始化加载器
db = Chroma.from_documents(split_docs, embeddings,persist_directory="./news_test")
# 持久化
db.persist()

db = Chroma(persist_directory="./news_test", embedding_function=embeddings)


In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(split_docs, embeddings)
db.save_local("./news_test")

db = FAISS.load_local("./news_test",embeddings=embeddings)

In [ ]:
from langchain.chains import RetrievalQA
import IPython

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
# 进行问答
query = "2022年腾讯营收多少"
print(qa.run(query))

In [ ]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
import re

class TfboyLLM(LLM):

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        print("问题:",prompt)
        pattern = re.compile(r'^.*(\d+[*/+-]\d+).*$')
        match = pattern.search(prompt)
        if match:
            result = eval(match.group(1))
        elif "？" in prompt:
            rep_args = {"我":"你", "你":"我", "吗":"", "？":"！"}
            result = [(rep_args[c] if c in rep_args else c) for c in list(prompt)]
            result = ''.join(result)
        else:
            result = "很抱歉，请换一种问法。比如：1+1等于几"
        return result

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}



In [ ]:
llm = TfboyLLM()
print("答案:",llm("我能问你问题吗？"))

# 实例

## 信息抽取

In [6]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel



class CustomLLM(LLM):
    def __init__(self, model_name, quantization_bit=4):
        print('----1', model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        print('----2', model_name)
        self.model = AutoModel.from_pretrained(model_name, trust_remote_code=True).half().cuda().eval()
        # self.model = self.model.quantize(quantization_bit)

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        response, history = self.model.chat(self.tokenizer, prompt)
        return response, history

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}


In [7]:
llm = CustomLLM(model_name="THUDM/chatglm-6B-int4")
llm("This is a foobar thing")

----1 THUDM/chatglm-6B-int4


ValueError: ignored

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



text = "阿尔茨海默病(Alzheimer's disease, AD),俗称老年痴呆症,是一种全身性神经退行性疾病，它是由大脑神经退行性变性引起的，\
主要表现为记忆力减退、思维能力减退、行为变化等。阿尔茨海默病的原因尚不十分清楚，但是研究表明，阿尔茨海默病可能与遗传因素、环境因素、\
营养不良、压力过大、不良生活习惯等有关。根据世界卫生组织的统计数据，全球有超过4700万人患有阿尔茨海默病，其中美国有超过600万人患有阿尔茨海默病，\
欧洲有超过1000万人患有阿尔茨海默病，亚洲有超过2500万人患有阿尔茨海默病，其中中国有超过1000万人患有阿尔茨海默病。阿尔茨海默病的发病率与年龄有关，\
随着年龄的增长而增加，65岁以上的人群为主要受害群体，占比高达80%，其中45-65岁的人群占比为15%，20-45岁的人群占比为5%。65岁以上的人群发病率约为10%，\
75岁以上的人群发病率约为20%，85岁以上的人群发病率约为30%。根据统计，男性患病率高于女性，男性患病比例为1.4：1，即男性患病率比女性高出40%。\
根据统计，阿尔茨海默病在不同的人种中分布情况也有所不同。白人患病率最高，占总患病率的70%，黑人患病率次之，占总患病率的20%，\
其他少数民族患病率最低，占总患病率的10%。阿尔茨海默病在不同的饮食习惯中分布情况也有所不同。维生素B12缺乏的人群患病率更高，\
而均衡膳食的人群患病率较低。阿尔茨海默病不仅会给患者带来记忆力减退、思维能力减退、行为变化等症状，还会给患者的家庭带来巨大的心理负担。\
因此，患者应尽快就医，及时进行治疗。治疗阿尔茨海默病的方法有药物治疗、行为治疗、认知行为治疗等，具体治疗方案要根据患者的具体情况而定。"

#创建模板
fact_extraction_prompt = PromptTemplate(
    input_variables=["text_input"],
    template="从下面的本文中提取关键事实。尽量使用文本中的统计数据来说明事实:\n\n {text_input}"
)

#定义chain
fact_extraction_chain = LLMChain(llm=llm, prompt=fact_extraction_prompt)
facts = fact_extraction_chain.run(text)
print(facts)

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="What is a good slogan for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("colorful socks")
print(f"Concatenated output:\n{concat_output}")


concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("colorful socks")


In [ ]:
text = "阿尔茨海默病(Alzheimer's disease, AD),俗称老年痴呆症,是一种全身性神经退行性疾病，它是由大脑神经退行性变性引起的，\
主要表现为记忆力减退、思维能力减退、行为变化等。阿尔茨海默病的原因尚不十分清楚，但是研究表明，阿尔茨海默病可能与遗传因素、环境因素、\
营养不良、压力过大、不良生活习惯等有关。根据世界卫生组织的统计数据，全球有超过4700万人患有阿尔茨海默病，其中美国有超过600万人患有阿尔茨海默病，\
欧洲有超过1000万人患有阿尔茨海默病，亚洲有超过2500万人患有阿尔茨海默病，其中中国有超过1000万人患有阿尔茨海默病。阿尔茨海默病的发病率与年龄有关，\
随着年龄的增长而增加，65岁以上的人群为主要受害群体，占比高达80%，其中45-65岁的人群占比为15%，20-45岁的人群占比为5%。65岁以上的人群发病率约为10%，\
75岁以上的人群发病率约为20%，85岁以上的人群发病率约为30%。根据统计，男性患病率高于女性，男性患病比例为1.4：1，即男性患病率比女性高出40%。\
根据统计，阿尔茨海默病在不同的人种中分布情况也有所不同。白人患病率最高，占总患病率的70%，黑人患病率次之，占总患病率的20%，\
其他少数民族患病率最低，占总患病率的10%。阿尔茨海默病在不同的饮食习惯中分布情况也有所不同。维生素B12缺乏的人群患病率更高，\
而均衡膳食的人群患病率较低。阿尔茨海默病不仅会给患者带来记忆力减退、思维能力减退、行为变化等症状，还会给患者的家庭带来巨大的心理负担。\
因此，患者应尽快就医，及时进行治疗。治疗阿尔茨海默病的方法有药物治疗、行为治疗、认知行为治疗等，具体治疗方案要根据患者的具体情况而定。"


#创建模板
fact_extraction_prompt = PromptTemplate(
    input_variables=["query"],
    template="从下面的本文中提取关键事实。尽量使用文本中的统计数据来说明事实:\n\n {query}"
)

fact_extraction_chain = DemoChain(llm=llm, prompt=fact_extraction_prompt, history=[])
response, history = fact_extraction_chain.run(query=text, history=[])
print(response, history)

In [ ]:
doctor_prompt = PromptTemplate(
    input_variables=["query"],
    template="你是神经内科医生。根据以下阿尔茨海默病的事实统计列表，为您的病人写一个简短的预防阿尔茨海默病的建议。 不要遗漏关键信息：\n\n {query}"
)

doctor_chain = DemoChain(llm=llm, prompt=doctor_prompt, history=history)
response, history = doctor_chain.run(query=response, history=history)
print(response, history)

In [ ]:
from langchain.chains import SimpleSequentialChain, SequentialChain


#定义SimpleSequentialChain
full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, doctor_chain], verbose=True)
response, history = full_chain.run(text)
print(response, history)
